In [3]:
import numpy as np
import pandas as pd
import time

import random

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
from tqdm import tqdm

import pickle5 as pickle

import matplotlib.ticker as tick

import sys
sys.path.append('..')

from nsgp_vi import nsgpVI
from nsgp_vi_3ns_params import nsgp_vi_3ns_params

# We'll use double precision throughout for better numerics.
dtype = np.float64

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

plt.style.use('ggplot') 
plt.style.use('seaborn-paper') 
plt.style.use('seaborn-whitegrid') 

In [ ]:
#plot the synthetic data for 1 individual, and the lengthscale and the amplitude, the mean is stationary and set at 0

df = pd.read_csv('../data/ns_synthetic_data_indv_1_0.csv')
T = df['Time'].values[:,None]
X = np.array(df['observations']).reshape(len(T),1)

true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)

#mean 0, n1

f, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(40,10))

ax1.plot(T,X,color='C1')
ax1.set_xlabel('Time',size=35)
ax1.set_ylabel('Observations',size=35)
ax1.text(-0.05,1,'A', size=35, transform=ax1.transAxes)
ax1.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax2.plot(T,true_len,color='C1')
ax2.set_xlabel('Time',size=35)
ax2.set_ylabel('Lengthscale',size=35)
ax2.text(-0.05,1,'B', size=35, transform=ax2.transAxes)
ax2.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax3.plot(T,true_var,color='C1')
ax3.set_xlabel('Time',size=35)
ax3.set_ylabel('Amplitude',size=35)
ax3.text(-0.05,1,'C', size=35, transform=ax3.transAxes)
ax3.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

f.tight_layout()

plt.savefig("Sythetic_dataset_vi_A.pdf",dpi=600)

In [ ]:
# plot the data when the mean is non-stationary 

#ns mean 

df = pd.read_csv('../data/ns_synthetic_data_mean_indv_1_4.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
true_mean = np.array(df['Mean']).reshape(len(T),1)

#mean 0, n1_4

f, (ax1, ax2,ax3,ax4) = plt.subplots(1,4,figsize=(40,10))

ax1.plot(T,X,color='C1')
ax1.set_xlabel('Time',size=35)
ax1.set_ylabel('Observations',size=35)
ax1.text(-0.05,1,'A', size=35, transform=ax1.transAxes)
ax1.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax2.plot(T,true_len,color='C1')
ax2.set_xlabel('Time',size=35)
ax2.set_ylabel('Lengthscale',size=35)
ax2.text(-0.08,1,'B', size=35, transform=ax2.transAxes)
ax2.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax3.plot(T,true_var,color='C1')
ax3.set_xlabel('Time',size=35)
ax3.set_ylabel('Amplitude',size=35)
ax3.text(-0.08,1,'C', size=35, transform=ax3.transAxes)
ax3.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax4.plot(T,true_mean,color='C1')
ax4.set_xlabel('Time',size=35)
ax4.set_ylabel('Mean',size=35)
ax4.text(-0.08,1,'D', size=35, transform=ax4.transAxes)
ax4.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

f.tight_layout()

plt.savefig("Sythetic_dataset_vi_B.pdf",dpi=600)

In [4]:
       # two latent functions scenario, mean is not a GP
       
        df = pd.read_csv('../data/ns_synthetic_data_indv_128_0.csv')
        T = df['Time'].values[:,None]
        ID = df['ID'].values
        X = np.array(df['observations']).reshape(len(T),1)
        true_len = np.array(df['Lengthscale']).reshape(len(T),1)
        true_var = np.array(df['Variance']).reshape(len(T),1)
        
        num_training_points_ = T.shape[0]
        num_inducing_points_ = 50
        inducing_index_points = np.linspace(0., 60*24., num_inducing_points_, endpoint=False)[..., np.newaxis]
        np.random.shuffle(inducing_index_points)
        
        BATCH_SIZE=8
        SEG_LENGTH=1024#decrease to 500 when there are three latent parameters
        allT = []
        allX = []
        
        for i in np.unique(df['ID'].values):
            allT.append(df['Time'][df['ID'].values == i].values[...,None])
            allX.append(df['observations'][df['ID'].values == i].values[...,None])

        class segment_generator:
            def __iter__(self):
                # loop over individuals
                self.i = 0
                self.max_i = len(allT)
                
                # loop over segments
                self.j = 0
                self.max_j = num_training_points_//(self.max_i*SEG_LENGTH)
            
                return self
            
            def __next__(self):
                
                if self.i == self.max_i:
                    self.i = 0
                    self.j +=1
                    if self.j==self.max_j:
                        raise StopIteration
                
                T = allT[self.i]
                X = allX[self.i] 

                TT = T[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
                XX = X[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
        
                self.i += 1

                return TT,XX
            
        dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
        dataset = dataset.map(lambda dd: (dd[0],dd[1]))
        dataset = dataset.shuffle(1024)
        dataset = dataset.batch(BATCH_SIZE)
        
        kernel_len_a = tfp.util.TransformedVariable(2.0, tfb.Softplus(),dtype=tf.float64, name='k_len_a',trainable=True)
        kernel_len_l = tfp.util.TransformedVariable(30.0,tfb.Chain([tfb.Scale(np.float64(10.)),tfb.Softplus()]),dtype=tf.float64, name='k_len_l',trainable=True)
        
        # amplitude kernel parameters, lower levels
        kernel_amp_a = tfp.util.TransformedVariable(2.0, tfb.Softplus(), dtype=tf.float64, name='k_amp_a',trainable=True)
        kernel_amp_l = tfp.util.TransformedVariable(30.0,tfb.Chain([tfb.Scale(np.float64(10.)),tfb.Softplus()]), dtype=tf.float64, name='k_amp_l',trainable=True)
        
        #kernels on the second layer
        kernel_len = tfk.ExponentiatedQuadratic(kernel_len_a,kernel_len_l)
        kernel_amp = tfk.ExponentiatedQuadratic(kernel_amp_a,kernel_amp_l)
        
        #print(str(nrd))

        vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  
        
         

In [ ]:
# three latent function scenario, mean is modelled by a GP

#lengthscale kernel parameters,lower levels
kernel_len_a = tfp.util.TransformedVariable(2.0, tfb.Softplus(),dtype=tf.float64, name='k_len_a',trainable=True)
kernel_len_l = tfp.util.TransformedVariable(30.0, tfb.Chain([tfb.Shift(np.float64(10.0)), tfb.Softplus()]),dtype=tf.float64, name='k_len_l',trainable=True)

# amplitude kernel parameters, lower levels
kernel_amp_a = tfp.util.TransformedVariable(2.0, tfb.Softplus(),dtype=tf.float64, name='k_amp_a',trainable=True)
kernel_amp_l = tfp.util.TransformedVariable(30.0, tfb.Chain([tfb.Shift(np.float64(10.0)), tfb.Softplus()]),dtype=tf.float64, name='k_amp_l',trainable=True)

# mean kernel parameters, lower levels
kernel_mean_a = tfp.util.TransformedVariable(2.0, tfb.Softplus(),dtype=tf.float64, name='k_mean_a',trainable=True)
kernel_mean_l = tfp.util.TransformedVariable(30.0, tfb.Chain([tfb.Shift(np.float64(10.)), tfb.Softplus()]),dtype=tf.float64, name='k_mean_l',trainable=True)

#kernels on the second layer
kernel_len = tfk.ExponentiatedQuadratic(kernel_len_a,kernel_len_l)
kernel_amp = tfk.ExponentiatedQuadratic(kernel_amp_a,kernel_amp_l)
kernel_mean = tfk.ExponentiatedQuadratic(kernel_mean_a,kernel_mean_l)

#added kernel_mean
vgp = nsgp_vi_3ns_params(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [5]:
vgp.trainable_variables

In [ ]:
# when optimising
loss = vgp.optimize(BATCH_SIZE, SEG_LENGTH, NUM_EPOCHS=1000)


In [7]:
#plot all the plots at once

# stationary model mean, and the mean of the data is stationary and set at 0.

ZZ = np.linspace(0,24*60,200)[:,None]

fig, ax = plt.subplots(3, 2,figsize=(30,33))

ax00 = ax[0,0]
ax01= ax[0,1]

ax10 = ax[1,0]
ax11= ax[1,1]

ax20 = ax[2,0]
ax21= ax[2,1]

#n1

#load the data for n1, choose one of five

df = pd.read_csv('../data/ns_synthetic_data_indv_1_0.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../results/T_ind_n1_0.npy')

#load the model
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=None,num_training_points=num_training_points_, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#load the parameters

with open('../results/opt_n1_0.pkl', 'rb') as f:
    loadp = pickle.load(f)

for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)

#plot for n1    
[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5
    

ax00.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax00.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=1.0)

ax00.set_xlabel('Time',size=30)
ax00.set_ylabel('Lengthscale',size=30)
ax00.text(-0.05,1,'A', size=40, transform=ax[0, 0].transAxes)
ax00.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)


ax01.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=1.0)
ax01.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax01.set_xlabel('Time',size=30)
ax01.set_ylabel('Amplitude',size=30)
ax01.text(-0.05,1,'B', size=40, transform=ax[0, 1].transAxes)
ax01.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

#n8

#load the data

df = pd.read_csv('../data/ns_synthetic_data_indv_8_2.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)

num_training_points = T.shape[0]


#load the inducing points
inducing_index_points = np.load('../results/T_ind_n8_2.npy')

#load the model
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=None,num_training_points=num_training_points, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#load the parameters

with open('../results/opt_n8_2.pkl', 'rb') as f:
    loadp = pickle.load(f)

for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)

#plot for n8
[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax10.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax10.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=1.0)

ax10.set_xlabel('Time',size=30)
ax10.set_ylabel('Lengthscale',size=30)
ax10.text(-0.07,1,'C', size=40, transform=ax[1, 0].transAxes)
ax10.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

ax11.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=1.0)
ax11.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax11.set_xlabel('Time',size=30)
ax11.set_ylabel('Amplitude',size=30)
ax11.text(-0.05,1,'D', size=40, transform=ax[1, 1].transAxes)
ax11.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

#n128 data

#load the data

df = pd.read_csv('../data/ns_synthetic_data_indv_128_3.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
num_training_points = T.shape[0]

#load the inducing points
inducing_index_points = np.load('../results/T_ind_n128_3.npy')   

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=None,num_training_points=num_training_points, num_sequential_samples=5,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#load the parameters

with open('../results/opt_n128_3.pkl', 'rb') as f:
    loadp = pickle.load(f)    
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)   

#plot for n128
[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])
    
len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5    
    
ax20.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax20.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=1.0)

ax20.set_xlabel('Time',size=30)
ax20.set_ylabel('Lengthscale',size=30)
ax20.text(-0.07,1,'E', size=40, transform=ax[2, 0].transAxes)
ax20.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

ax21.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=1.0)
ax21.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax21.set_xlabel('Time',size=30)
ax21.set_ylabel('Amplitude',size=30)
ax21.text(-0.05,1,'F', size=40, transform=ax[2, 1].transAxes)
ax21.tick_params(axis='both', which='major', labelsize=30)
tick_locator = tick.MaxNLocator(nbins=7)

fig.tight_layout()

plt.savefig("Sythetic_inference_vi.pdf",dpi=600)

plt.show()


In [ ]:
# create the 3x3 plot for ns mean 

#plot all the plots at once
ZZ = np.linspace(0,24*60,200)[:,None]

#fig, ax = plt.subplots(3, 3,figsize=(40,10))
fig, ax = plt.subplots(3, 3,figsize=(30,30))


ax00 = ax[0,0]
ax01= ax[0,1]
ax02= ax[0,2]

ax10 = ax[1,0]
ax11= ax[1,1]
ax12= ax[1,2]

ax20 = ax[2,0]
ax21= ax[2,1]
ax22= ax[2,2]

#n1

#load the data for n1

df = pd.read_csv('../data/ns_synthetic_data_mean__indv_1_4.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
true_mean = np.array(df['Mean']).reshape(len(T),1)

num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../results/upd-syn-inf-n1-ns-mean-index-points.npy')

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#Load the inducing points and the optimized parameters 
with open('../results/upd-syn-inf-n1-ns-mean-param.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)
    
[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax00.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax00.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=0.5)


ax00.set_xlabel('Time',size=35)
ax00.set_ylabel('Lengthscale',size=35)
ax00.text(-0.10,1,'A', size=35, transform=ax00.transAxes)
ax00.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax01.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)
ax01.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=0.5)


ax01.set_xlabel('Time',size=35)
ax01.set_ylabel('Amplitude',size=35)
ax01.text(-0.10,1,'B', size=35, transform=ax01.transAxes)
ax01.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax02.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax02.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax02.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax02.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax02.plot(T[:8192],true_mean[:8192],'--',markersize=1,color='k',alpha=0.5)

ax02.set_xlabel('Time',size=35)
ax02.set_ylabel('Mean',size=35)
ax02.text(-0.10,1,'C', size=35, transform=ax02.transAxes)
ax02.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)


#n8

#load the data for n8

df = pd.read_csv('../data/ns_synthetic_data_mean_indv_8_4.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
true_mean = np.array(df['Mean']).reshape(len(T),1)

num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../results/upd-syn-inf-n8-ns-mean-index-points.npy')

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#Load the inducing points and the optimized parameters 
with open('../input/results/upd-syn-inf-n8-ns-mean-param.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)
    
[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax10.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax10.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=0.5)


ax10.set_xlabel('Time',size=35)
ax10.set_ylabel('Lengthscale',size=35)
ax10.text(-0.10,1,'D', size=35, transform=ax10.transAxes)
ax10.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax11.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)
ax11.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=0.5)


ax11.set_xlabel('Time',size=35)
ax11.set_ylabel('Amplitude',size=35)
ax11.text(-0.10,1,'E', size=35, transform=ax11.transAxes)
ax11.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax12.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax12.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax12.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax12.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax12.plot(T[:8192],true_mean[:8192],'--',markersize=1,color='k',alpha=0.5)

ax12.set_xlabel('Time',size=35)
ax12.set_ylabel('Mean',size=35)
ax12.text(-0.10,1,'F', size=35, transform=ax12.transAxes)
ax12.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)


#n128

#load the data for n128

df = pd.read_csv('../data/ns_synthetic_data_mean_indv_128_4.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)
true_mean = np.array(df['Mean']).reshape(len(T),1)

num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../results/upd-syn-inf-n128-ns-mean-index-points.npy')

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#Load the inducing points and the optimized parameters 
with open('../results/upd-syn-inf-n128-ns-mean-param.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)
    
[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax20.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax20.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=0.5)


ax20.set_xlabel('Time',size=35)
ax20.set_ylabel('Lengthscale',size=35)
ax20.text(-0.10,1,'G', size=35, transform=ax20.transAxes)
ax20.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax21.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)
ax21.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=0.5)


ax21.set_xlabel('Time',size=35)
ax21.set_ylabel('Amplitude',size=35)
ax21.text(-0.10,1.05,'H', size=35, transform=ax21.transAxes)
ax21.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax22.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax22.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax22.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax22.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax22.plot(T[:8192],true_mean[:8192],'--',markersize=1,color='k',alpha=0.5)

ax22.set_xlabel('Time',size=35)
ax22.set_ylabel('Mean',size=35)
ax22.text(-0.10,1,'I', size=35, transform=ax22.transAxes)
ax22.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

fig.tight_layout()

plt.savefig("Sythetic_inference_ns_mean_vi.pdf",dpi=600)

plt.show()

In [ ]:
# create the 3x4 plot for mean-0

#plot all the plots at once
ZZ = np.linspace(0,24*60,200)[:,None]

#fig, ax = plt.subplots(3, 3,figsize=(40,10))
fig, ax = plt.subplots(3, 4,figsize=(40,30))


ax00 = ax[0,0]
ax01= ax[0,1]
ax02= ax[0,2]
ax03= ax[0,3]


ax10 = ax[1,0]
ax11= ax[1,1]
ax12= ax[1,2]
ax13= ax[1,3]


ax20 = ax[2,0]
ax21= ax[2,1]
ax22= ax[2,2]
ax23= ax[2,3]


#n1

#load the data for n1

df = pd.read_csv('../input/vinsgpdata/ns_synthetic_data_indv_1_0.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)

num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../input/synthetic-0-mean-results/upd-syn-inf-n1-0-mean-index-points.npy')

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#Load the inducing points and the optimized parameters 
with open('../input/synthetic-0-mean-results/upd-syn-inf-n1-0-mean-param.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)
    
[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax00.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax00.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax00.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=0.5)


ax00.set_xlabel('Time',size=35)
ax00.set_ylabel('Lengthscale',size=35)
ax00.text(-0.10,1,'A', size=35, transform=ax00.transAxes)
ax00.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax01.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax01.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)
ax01.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=0.5)


ax01.set_xlabel('Time',size=35)
ax01.set_ylabel('Amplitude',size=35)
ax01.text(-0.10,1,'B', size=35, transform=ax01.transAxes)
ax01.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax02.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax02.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax02.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax02.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax02.plot(T[:8192],np.zeros((8192,1)),'--',markersize=1,color='k',alpha=0.5)


ax02.set_xlabel('Time',size=35)
ax02.set_ylabel('Mean',size=35)
ax02.text(-0.10,1.05,'C', size=35, transform=ax02.transAxes)
ax02.tick_params(axis='both', which='major', labelsize=35)
ax02.set_ylim([-0.5, 3])
tick_locator = tick.MaxNLocator(nbins=7)

ax03.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax03.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax03.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax03.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax03.plot(T[:8192],np.zeros((8192,1)),'--',markersize=1,color='k',alpha=0.5)


ax03.set_xlabel('Time',size=35)
ax03.set_ylabel('Mean',size=35)
ax03.text(-0.10,1,'D', size=35, transform=ax03.transAxes)
ax03.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)


#n8

#load the data for n8

df = pd.read_csv('../input/vinsgpdata/ns_synthetic_data_indv_8_2.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)

num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../input/synthetic-0-mean-results/upd-syn-inf-n8-0-mean-index-points.npy')

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#Load the inducing points and the optimized parameters 
with open('../input/synthetic-0-mean-results/upd-syn-inf-n8-0-mean-param.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)
    
[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax10.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax10.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax10.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=0.5)


ax10.set_xlabel('Time',size=35)
ax10.set_ylabel('Lengthscale',size=35)
ax10.text(-0.10,1,'E', size=35, transform=ax10.transAxes)
ax10.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax11.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax11.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)
ax11.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=0.5)


ax11.set_xlabel('Time',size=35)
ax11.set_ylabel('Amplitude',size=35)
ax11.text(-0.10,1,'F', size=35, transform=ax11.transAxes)
ax11.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax12.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax12.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax12.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax12.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax12.plot(T[:8192],np.zeros((8192,1)),'--',markersize=1,color='k',alpha=0.5)


ax12.set_xlabel('Time',size=35)
ax12.set_ylabel('Mean',size=35)
ax12.text(-0.10,1.05,'G', size=35, transform=ax12.transAxes)
ax12.tick_params(axis='both', which='major', labelsize=35)
ax12.set_ylim([-0.5, 3])
tick_locator = tick.MaxNLocator(nbins=7)

ax13.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax13.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax13.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax13.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax13.plot(T[:8192],np.zeros((8192,1)),'--',markersize=1,color='k',alpha=0.5)

ax13.set_xlabel('Time',size=35)
ax13.set_ylabel('Mean',size=35)
ax13.text(-0.10,1,'H', size=35, transform=ax13.transAxes)
ax13.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)



#n128

#load the data for n128

df = pd.read_csv('../input/vinsgpdata/ns_synthetic_data_indv_128_3.csv')
T = df['Time'].values[:,None]
ID = df['ID'].values
X = np.array(df['observations']).reshape(len(T),1)
true_len = np.array(df['Lengthscale']).reshape(len(T),1)
true_var = np.array(df['Variance']).reshape(len(T),1)

num_training_points_ = T.shape[0]


#Load the inducing points
inducing_index_points_ = np.load('../input/synthetic-0-mean-results/upd-syn-inf-n128-0-mean-index-points.npy')

#load the model 
vgp = nsgpVI(kernel_len,kernel_amp,kernel_mean,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points_,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  

#Load the inducing points and the optimized parameters 
with open('../input/synthetic-0-mean-results/upd-syn-inf-n128-0-mean-param.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)
    
[mean_mean,len_mean,amp_mean], [mean_var,len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

mean_mean = mean_mean[0,:,0].numpy()
mean_std = mean_var[:,0].numpy()**0.5


len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5

ax20.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax20.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)
ax20.plot(T[:8192],true_len[:8192],'--',markersize=1,color='k',alpha=0.5)


ax20.set_xlabel('Time',size=35)
ax20.set_ylabel('Lengthscale',size=35)
ax20.text(-0.10,1.05,'I', size=35, transform=ax20.transAxes)
ax20.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax21.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax21.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)
ax21.plot(T[:8192],true_var[:8192],'--',markersize=1,color='k',alpha=0.5)


ax21.set_xlabel('Time',size=35)
ax21.set_ylabel('Amplitude',size=35)
ax21.text(-0.10,1.05,'J', size=35, transform=ax21.transAxes)
ax21.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)

ax22.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax22.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax22.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax22.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax22.plot(T[:8192],np.zeros((8192,1)),'--',markersize=1,color='k',alpha=0.5)


ax22.set_xlabel('Time',size=35)
ax22.set_ylabel('Mean',size=35)
ax22.text(-0.10,1.05,'K', size=35, transform=ax22.transAxes)
ax22.tick_params(axis='both', which='major', labelsize=35)
ax22.set_ylim([-0.5, 2])
tick_locator = tick.MaxNLocator(nbins=7)

ax23.plot(ZZ,vgp.mean_mean + mean_mean,color='C1')
ax23.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.28*mean_std,vgp.mean_mean + mean_mean + 1.28*mean_std,color='C1',alpha=0.25)
ax23.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 1.96*mean_std,vgp.mean_mean + mean_mean + 1.96*mean_std,color='C1',alpha=0.25)
ax23.fill_between(ZZ[:,0],vgp.mean_mean + mean_mean - 2.58*mean_std,vgp.mean_mean + mean_mean + 2.58*mean_std,color='C1',alpha=0.25)
ax23.plot(T[:8192],np.zeros((8192,1)),'--',markersize=1,color='k',alpha=0.5)

ax23.set_xlabel('Time',size=35)
ax23.set_ylabel('Mean',size=35)
ax23.text(-0.10,1.05,'L', size=35, transform=ax23.transAxes)
ax23.tick_params(axis='both', which='major', labelsize=35)
tick_locator = tick.MaxNLocator(nbins=7)


fig.tight_layout()

plt.savefig("Sythetic_inference_0_mean_vi.pdf",dpi=300)

plt.show()